In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

#Torchtext is a powerful library that is useful for language preprocessing

from torchtext.legacy.datasets import Multi30k  # German -> English dataset
from torchtext.legacy.data import Field, BucketIterator 

# Field: Specify how preprocessing has to be done
# BucketIterator: iterator to do batching and padding

import spacy  # Tokenizer
import numpy as np

import random
import math
import time

from torch.utils.tensorboard import SummaryWriter   # to print to tensorboard

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
!python -m spacy download en_core_web_sm
spacy_encoder = spacy.load('en_core_web_sm')

     |████████████████████████████████| 12.0 MB 4.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
!python -m spacy download de_core_news_sm
spacy_decoder = spacy.load('de_core_news_sm')

     |████████████████████████████████| 14.9 MB 3.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [5]:
def tokenizer_en(text):
    return [tok.text for tok in spacy_encoder.tokenizer(text)]
#'Hello my name is Lark'  ->  ['Hello', 'my', 'name' 'is', 'Lark']

def tokenizer_de(text):
    return [tok.text for tok in spacy_decoder.tokenizer(text)]

In [6]:
source = Field(tokenize = tokenizer_en, lower = True,
              init_token = '<sos>', eos_token = '<eos>')

target = Field(tokenize = tokenizer_de, lower = True,
              init_token = '<sos>', eos_token = '<eos>')

In [7]:
train_data, validation_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                         fields = (source, target))

downloading training.tar.gz


100%|██████████| 1.21M/1.21M [00:03<00:00, 302kB/s]


downloading validation.tar.gz


100%|██████████| 46.3k/46.3k [00:00<00:00, 92.1kB/s]


downloading mmt_task1_test2016.tar.gz


100%|██████████| 66.2k/66.2k [00:00<00:00, 88.3kB/s]


In [8]:
#Making sure data is correctly loaded
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(validation_data)}')
print(f'Number of test examples: {len(test_data)}')

Number of training examples: 29000
Number of validation examples: 1014
Number of test examples: 1000


In [9]:
print(vars(train_data.examples[0]))

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'i', 'm', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [10]:
source.build_vocab(train_data, min_freq = 2)
target.build_vocab(train_data, min_freq = 2)

print(f'Tokens in source (de) vocabulary: {len(source.vocab)}')
print(f'Tokens in target (en) vocabulary: {len(target.vocab)}')

Tokens in source (de) vocabulary: 7873
Tokens in target (en) vocabulary: 5923


In [11]:
device = 'cpu'
if torch.cuda.is_available(): 
    device = 'cuda'
device = torch.device(device)

In [12]:
BATCH_SIZE = 128
train_iterator, validation_iterator, test_iterator = BucketIterator.splits(
                                                            (train_data, validation_data, test_data), 
                                                            batch_size = BATCH_SIZE, 
                                                            device = device)

In [13]:
# Building the encoder
class Encoder(nn.Module):

    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()   # boilerplate code

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    def forward(self, X):
        # X.shape = (Tx, m)

        embedding = self.dropout(self.embedding(X))
        # embedding.shape = (Tx, m, embedding_size)
        outputs, (hidden, cell) = self.rnn(embedding)

        return hidden, cell

In [14]:
# Building the decoder
class Decoder(nn.Module):

    def __init__(self, output_size, embedding_size, hidden_size, num_layers, p):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_size, embedding_size)
        self.dropout = nn.Dropout(p)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X, hidden, cell):
        # X.shape = (m), but we want (1, m)
        X = X.unsqueeze(0)

        embedding = self.dropout(self.embedding(X))
        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))

        pred = self.fc(outputs.squeeze(0))
        # pred.shape = (1, m, vocab_length) -> (m, vocan_length)

        return pred, hidden, cell

In [15]:
# Building our Seq2Seq model
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hidden_size == decoder.hidden_size, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_size

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)
        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio
            most_probable = output.argmax(1)

            input = trg[t] if teacher_force else most_probable
        
        return outputs

In [16]:
# Training the Seq2Seq Model

INPUT_SIZE = len(source.vocab)    # Length of one-hot vectors for German
OUTPUT_SIZE = len(target.vocab)   # Length of one-hot vectors for German

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_SIZE, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_SIZE, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [17]:
def initialize_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
# model.apply applies that function on all modules and submodules in the model
model.apply(initialize_weights)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7873, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(5923, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=5923, bias=True)
  )
)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,926,691 trainable parameters


In [19]:
optimizer = optim.Adam(model.parameters())

In [20]:
# Defining our loss function
# Our loss function calculates the average loss per token, however by passing the index of the <pad> token as the ignore_index argument we ignore the loss whenever the target token is a padding token.
padded_target = target.vocab.stoi[target.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = padded_target)

In [21]:
# Defining the training modules
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        X = batch.src
        y = batch.trg

        optimizer.zero_grad()
        output = model(X, y)

        output_size = output.shape[-1]

        output = output[1:].view(-1, output_size)
        y = y[1:].view(-1)

        loss = criterion(output, y)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [22]:
# Defining the model evaluation function
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [23]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, validation_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 13s
	Train Loss: 4.596 | Train PPL:  99.053
	 Val. Loss: 4.905 |  Val. PPL: 134.942
Epoch: 02 | Time: 1m 13s
	Train Loss: 4.264 | Train PPL:  71.077
	 Val. Loss: 4.610 |  Val. PPL: 100.444
Epoch: 03 | Time: 1m 13s
	Train Loss: 4.048 | Train PPL:  57.299
	 Val. Loss: 4.465 |  Val. PPL:  86.957
Epoch: 04 | Time: 1m 13s
	Train Loss: 3.894 | Train PPL:  49.110
	 Val. Loss: 4.355 |  Val. PPL:  77.857
Epoch: 05 | Time: 1m 13s
	Train Loss: 3.754 | Train PPL:  42.712
	 Val. Loss: 4.240 |  Val. PPL:  69.413
Epoch: 06 | Time: 1m 13s
	Train Loss: 3.638 | Train PPL:  38.022
	 Val. Loss: 4.105 |  Val. PPL:  60.620
Epoch: 07 | Time: 1m 13s
	Train Loss: 3.501 | Train PPL:  33.146
	 Val. Loss: 4.015 |  Val. PPL:  55.439
Epoch: 08 | Time: 1m 13s
	Train Loss: 3.385 | Train PPL:  29.523
	 Val. Loss: 4.005 |  Val. PPL:  54.882
Epoch: 09 | Time: 1m 12s
	Train Loss: 3.282 | Train PPL:  26.642
	 Val. Loss: 3.954 |  Val. PPL:  52.155
Epoch: 10 | Time: 1m 13s
	Train Loss: 3.189 | Train PPL

In [27]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.881 | Test PPL:  48.481 |
